<a href="https://colab.research.google.com/github/patakib/weather_analysis_RNN/blob/main/weather_analysis_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from google.colab import files

In [ ]:
raw = files.upload()



TypeError: ignored

In [ ]:
rawdf = pd.read_csv('weatherHistory.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
rawdf.head()

In [ ]:
print("There are {} records and {} features in the dataset".format(rawdf.shape[0], rawdf.shape[1]))

In [ ]:
rawdf.info()

In [ ]:
import missingno as msno
msno.matrix(rawdf)

In [ ]:
rawdf.describe().transpose().round(2)

In [ ]:
date_time = rawdf['Formatted Date'].apply(pd.to_datetime, utc=True)

In [ ]:
plot_cols = list(rawdf.columns.values)
plot_features = rawdf[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True, color=['darkorange','orange','darkblue','lightblue','purple','brown','green','pink'], figsize=(20,20))

In [ ]:
plot_features = rawdf[plot_cols][:24]
plot_features.index = date_time[:24]
_ = plot_features.plot(subplots=True, color=['darkorange','orange','darkblue','lightblue','purple','brown','green','pink'], figsize=(20,20))

In [ ]:
import plotly.express as px
df = px.data.wind()
df

In [ ]:
winddf = rawdf[['Wind Bearing (degrees)','Wind Speed (km/h)']]

In [ ]:
winddf.head()

In [ ]:
bins = [0,11.25,33.75,56.25,78.75,101.25,123.75,146.25,168.75,191.25,213.75,236.25,258.75,281.25,303.75,326.25,348.75]
names = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW']

winddf['Wind Bearing'] = pd.cut(winddf['Wind Bearing (degrees)'], bins, labels=names,ordered=True)

In [ ]:
winddf.head()

In [ ]:
max(winddf['Wind Speed (km/h)'])

In [ ]:
np.percentile(winddf['Wind Speed (km/h)'],25)

In [ ]:
speed_bins = [0,10,20,30,40,50,60,70]
speed_categories = ['0-10 km/h','11-20 km/h','21-30 km/h','31-40 km/h','41-50 km/h','51-60 km/h','60+ km/h']

winddf['Wind Speed Categories'] = pd.cut(winddf['Wind Speed (km/h)'], speed_bins, labels=speed_categories)

In [ ]:
winddf.head()

In [ ]:
winddf_grouped = winddf.groupby(by=['Wind Bearing','Wind Speed Categories']).agg({'Wind Speed (km/h)': ['count']}).reset_index()

In [ ]:
winddf_grouped.columns = ['Wind Bearing','Wind Speed Categories','count']

In [ ]:
winddf_grouped.head()

In [ ]:
import plotly.express as px
fig = px.bar_polar(winddf_grouped, r="count", theta="Wind Bearing",
                   color="Wind Speed Categories", template="plotly",
                   color_discrete_sequence= px.colors.sequential.haline)
fig.show()

In [ ]:
rawdf['Precip Type'].unique()

In [ ]:
dummydf = pd.get_dummies(rawdf, columns=['Summary','Precip Type'])

In [ ]:
dummydf.head()

In [ ]:
print("There are {} records and {} features in the dataset".format(dummydf.shape[0], dummydf.shape[1]))

In [ ]:
df = dummydf.copy()

In [84]:
df.head()

,Formatted Date,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary,Summary_Breezy,Summary_Breezy and Dry,Summary_Breezy and Foggy,Summary_Breezy and Mostly Cloudy,Summary_Breezy and Overcast,Summary_Breezy and Partly Cloudy,Summary_Clear,Summary_Dangerously Windy and Partly Cloudy,Summary_Drizzle,Summary_Dry,Summary_Dry and Mostly Cloudy,Summary_Dry and Partly Cloudy,Summary_Foggy,Summary_Humid and Mostly Cloudy,Summary_Humid and Overcast,Summary_Humid and Partly Cloudy,Summary_Light Rain,Summary_Mostly Cloudy,Summary_Overcast,Summary_Partly Cloudy,Summary_Rain,Summary_Windy,Summary_Windy and Dry,Summary_Windy and Foggy,Summary_Windy and Mostly Cloudy,Summary_Windy and Overcast,Summary_Windy and Partly Cloudy,Precip Type_rain,Precip Type_snow
0,2006-04-01 00:00:00.000 +0200,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,2006-04-01 01:00:00.000 +0200,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,2006-04-01 02:00:00.000 +0200,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
3,2006-04-01 03:00:00.000 +0200,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,2006-04-01 04:00:00.000 +0200,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [85]:
training_set = df.iloc[:,1:2]
training_set

,Temperature (C)
0,9.472222
1,9.355556
2,9.377778
3,8.288889
4,8.755556
...,...
96448,26.016667
96449,24.583333
96450,22.038889
96451,21.522222


In [86]:
#Feature Scaling 
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [87]:
x_train = []
y_train = []
n_future = 4 # next 4 days temperature forecast
n_past = 60 # Past 60 days 
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])     
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

In [89]:
x_train.shape

(96390, 60, 1)

In [92]:
from keras.models import Sequential
from keras.layers import LSTM,Dense ,Dropout, Bidirectional
# Fitting RNN to training set using Keras Callbacks. Read Keras callbacks docs for more info.

In [94]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=60, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 60 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 60 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 60))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
regressor.fit(x_train, y_train, epochs=5,batch_size=32 )

Epoch 1/5
3013/3013 [==============================] - 381s 124ms/step - loss: 0.0099 - acc: 0.3241
Epoch 2/5
3013/3013 [==============================] - 392s 130ms/step - loss: 0.0020 - acc: 0.5496
Epoch 3/5
3013/3013 [==============================] - 391s 130ms/step - loss: 0.0016 - acc: 0.6086
Epoch 4/5
3013/3013 [==============================] - 392s 130ms/step - loss: 0.0014 - acc: 0.6261
Epoch 5/5
3013/3013 [==============================] - 391s 130ms/step - loss: 0.0013 - acc: 0.6330


In [ ]:
# read test dataset
testdataset = df.iloc[:30]
#get only the temperature column
testdataset = testdataset.iloc[:30,3:4].values
real_temperature = pd.read_csv('data (12).csv')
real_temperature = real_temperature.iloc[30:,3:4].values
testing = sc.transform(testdataset)
testing = np.array(testing)
testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))